In [363]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook
import numpy as np
import pandas as pd

In [364]:
backdoor = pd.read_csv('csv/stl10/cv/b_train.csv',header=None)
backdoor = backdoor.values

In [365]:
backdoor[:, -1]

array([0., 0., 0., ..., 0., 0., 0.])

In [366]:
res_backdoor  = np.concatenate((backdoor, np.zeros((len(backdoor[:, -1]), 1))), axis=1)
print(len(res_backdoor))

6000


In [367]:
b_train

array([[19.67174   , -1.1232085 , -0.89969444, ..., -0.888966  ,
         0.        ,  0.        ],
       [24.00162   , -1.6951634 , -3.6819868 , ..., -2.3598003 ,
         0.        ,  0.        ],
       [19.522959  ,  3.7918465 , -5.430204  , ...,  0.47496593,
         0.        ,  0.        ],
       ...,
       [16.596119  , -1.4022186 , -5.1754313 , ..., -1.912952  ,
         0.        ,  0.        ],
       [23.648275  , -5.083376  , -7.2755322 , ..., -1.8304331 ,
         0.        ,  0.        ],
       [23.174734  , -4.316598  , -1.4159449 , ..., -0.3981294 ,
         0.        ,  0.        ]])

In [368]:
clean = pd.read_csv('csv/stl10/cv/c_train.csv',header=None)
clean = clean.values

In [369]:
res_clean  = np.concatenate((clean,  np.ones((len(clean[: ,-1]), 1))), axis=1)
print(len(res_clean))

6000


In [370]:
model_results = np.vstack((res_clean, res_backdoor))

In [371]:
model_results

array([[-1.3869573 ,  2.1415608 ,  9.037069  , ...,  6.801649  ,
         2.        ,  1.        ],
       [-6.4431252 , -8.644714  ,  0.94844633, ...,  2.1079938 ,
         4.        ,  1.        ],
       [-9.214293  ,  8.9453335 , -7.9739847 , ..., -3.933224  ,
         4.        ,  1.        ],
       ...,
       [ 9.524741  ,  1.8170424 , -4.6482253 , ..., -4.493582  ,
         0.        ,  0.        ],
       [10.513698  ,  2.4764946 , -3.4415903 , ..., -3.591676  ,
         0.        ,  0.        ],
       [11.277263  , -0.5133069 , -3.7015157 , ..., -5.871673  ,
         0.        ,  0.        ]])

In [372]:
model_results.shape

(12000, 12)

In [373]:
from sklearn.ensemble import RandomForestClassifier
from mblearn import AttackModels

In [374]:
rf_attack = RandomForestClassifier(n_estimators=100)
attacker = AttackModels(target_classes=10, attack_learner=rf_attack)
attacker.fit(model_results)

In [376]:
b_ci = pd.read_csv('b_train.csv',header=None)
c_ci = pd.read_csv('c_train.csv',header=None)
c_ci = c_ci.values
b_ci = b_ci.values

In [377]:
#clean_test = clean_ci[149850:, :]

In [378]:
backdoor_test_out = b_ci[:,:-1]
backdoor_test_label = b_ci[:,-1]
clean_test_out = c_ci[:,:-1]
clean_test_label = c_ci[:,-1]

In [379]:
backdoor_test_label = backdoor_test_label.astype(int)
clean_test_label = clean_test_label.astype(int)

In [380]:
clean_test_out.shape

(90690, 10)

In [381]:
clean_test_label.shape

(90690,)

In [383]:
mem_res_clean = attacker.predict(clean_test_out ,clean_test_label, batch=True)
mem_res_backdoor = attacker.predict(backdoor_test_out ,backdoor_test_label, batch=True)

In [384]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [388]:
y_pred = np.concatenate((np.argmax(mem_res_clean, axis=1), np.argmax(mem_res_backdoor, axis=1)))
y_true = np.concatenate((np.ones_like(clean_test_label), np.zeros_like(backdoor_test_label)))

In [389]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [390]:
accuracy_score(y_true, y_pred)

0.5

In [391]:
confusion_matrix(y_true, y_pred)

array([[90690,     0],
       [90690,     0]])

In [409]:
arr = y_pred.reshape(-1, 9990) 

for i, group in enumerate(arr):
    ones = np.sum(group == 1)
    zeros = np.sum(group == 0)

    print(f'model {i}:')
    print(f'Ones: {ones}')
    print(f'Zeros: {zeros}')

    if ones > zeros :
        print('Result: 1')
    else:  
        print('Result: 0')

model 0:
Ones: 892
Zeros: 9098
Result: 0
model 1:
Ones: 3183
Zeros: 6807
Result: 0
model 2:
Ones: 1034
Zeros: 8956
Result: 0
model 3:
Ones: 1086
Zeros: 8904
Result: 0
model 4:
Ones: 538
Zeros: 9452
Result: 0
model 5:
Ones: 628
Zeros: 9362
Result: 0
model 6:
Ones: 690
Zeros: 9300
Result: 0
model 7:
Ones: 898
Zeros: 9092
Result: 0
model 8:
Ones: 1412
Zeros: 8578
Result: 0
model 9:
Ones: 720
Zeros: 9270
Result: 0
model 10:
Ones: 0
Zeros: 9990
Result: 0
model 11:
Ones: 0
Zeros: 9990
Result: 0
model 12:
Ones: 0
Zeros: 9990
Result: 0
model 13:
Ones: 0
Zeros: 9990
Result: 0
model 14:
Ones: 0
Zeros: 9990
Result: 0
model 15:
Ones: 0
Zeros: 9990
Result: 0
model 16:
Ones: 0
Zeros: 9990
Result: 0
model 17:
Ones: 0
Zeros: 9990
Result: 0
model 18:
Ones: 0
Zeros: 9990
Result: 0
model 19:
Ones: 0
Zeros: 9990
Result: 0
